In [1]:
import findspark #cai dat findspark cho env
findspark.init()
findspark.find()
import pyspark

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.functions import sum
from pyspark.sql.types import * 
from pyspark.sql.functions import lit
import pyodbc
import pandas as pd
import pyspark.sql.functions as sf

In [3]:
spark = SparkSession.builder.config("spark.driver.memory", "6g").getOrCreate()

In [4]:
path = 'D:\\Bigdata\\Dataset\\'
file_name = '20220401.json'
date = '2022-04-01'

In [5]:
df = spark.read.json(path+file_name)
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- _index: string (nullable = true)
 |-- _score: long (nullable = true)
 |-- _source: struct (nullable = true)
 |    |-- AppName: string (nullable = true)
 |    |-- Contract: string (nullable = true)
 |    |-- Mac: string (nullable = true)
 |    |-- TotalDuration: long (nullable = true)
 |-- _type: string (nullable = true)



In [6]:
df.show()

+--------------------+-------+------+--------------------+-----+
|                 _id| _index|_score|             _source|_type|
+--------------------+-------+------+--------------------+-----+
|AX_momhia1FFivsGrn9o|history|     0|{KPLUS, HNH579912...|kplus|
|AX_momhca1FFivsGrnvg|history|     0|{KPLUS, HUFD40665...|kplus|
|AX_momhaa1FFivsGrnny|history|     0|{KPLUS, HNH572635...|kplus|
|AX_momhca1FFivsGrnvv|history|     0|{KPLUS, HND141717...|kplus|
|AX_momhia1FFivsGrn98|history|     0|{KPLUS, HNH743103...|kplus|
|AX_momg9a1FFivsGrnkS|history|     0|{KPLUS, HNH893773...|kplus|
|AX_momhca1FFivsGrnwA|history|     0|{KPLUS, HND083642...|kplus|
|AX_momhfa1FFivsGrn2u|history|     0|{KPLUS, DNFD74404...|kplus|
|AX_momhca1FFivsGrnwP|history|     0|{KPLUS, DTFD21200...|kplus|
|AX_momhca1FFivsGrnwU|history|     0|{KPLUS, LDFD05747...|kplus|
|AX_momhfa1FFivsGrn24|history|     0|{KPLUS, HNH063566...|kplus|
|AX_momhia1FFivsGrn-W|history|     0|{KPLUS, HNH866786...|kplus|
|AX_momhia1FFivsGrn-a|his

In [7]:
df1 = df.select('_source.Contract','_source.Appname','_source.TotalDuration')
print(df1.count())
print(df1.select('Contract').distinct().count())
df1.show()

1654519
1360622
+---------+-------+-------------+
| Contract|Appname|TotalDuration|
+---------+-------+-------------+
|HNH579912|  KPLUS|          254|
|HUFD40665|  KPLUS|         1457|
|HNH572635|  KPLUS|         2318|
|HND141717|  KPLUS|         1452|
|HNH743103|  KPLUS|          251|
|HNH893773|  KPLUS|          924|
|HND083642|  KPLUS|         1444|
|DNFD74404|  KPLUS|          691|
|DTFD21200|  KPLUS|         1436|
|LDFD05747|  KPLUS|         1434|
|HNH063566|  KPLUS|          687|
|HNH866786|  KPLUS|          248|
|NBAAA1128|  KPLUS|          247|
|HNH960439|  KPLUS|          683|
|HNJ035736|  KPLUS|          246|
|NTFD93673|  KPLUS|         2288|
|HNJ063267|  KPLUS|         2282|
|HNH790383|  KPLUS|          906|
|THFD12466|  KPLUS|          242|
|HNH566080|  KPLUS|          242|
+---------+-------+-------------+
only showing top 20 rows



In [8]:
df1 = df1.withColumn("Type",
                             when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
                            .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
                            .when((col("AppName") == 'RELAX'), "Giải Trí")
                            .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
                            .when((col("AppName") == 'SPORT'), "Thể Thao")
                            .otherwise("Error"))
df1.show()

+---------+-------+-------------+-----------+
| Contract|Appname|TotalDuration|       Type|
+---------+-------+-------------+-----------+
|HNH579912|  KPLUS|          254|Truyền Hình|
|HUFD40665|  KPLUS|         1457|Truyền Hình|
|HNH572635|  KPLUS|         2318|Truyền Hình|
|HND141717|  KPLUS|         1452|Truyền Hình|
|HNH743103|  KPLUS|          251|Truyền Hình|
|HNH893773|  KPLUS|          924|Truyền Hình|
|HND083642|  KPLUS|         1444|Truyền Hình|
|DNFD74404|  KPLUS|          691|Truyền Hình|
|DTFD21200|  KPLUS|         1436|Truyền Hình|
|LDFD05747|  KPLUS|         1434|Truyền Hình|
|HNH063566|  KPLUS|          687|Truyền Hình|
|HNH866786|  KPLUS|          248|Truyền Hình|
|NBAAA1128|  KPLUS|          247|Truyền Hình|
|HNH960439|  KPLUS|          683|Truyền Hình|
|HNJ035736|  KPLUS|          246|Truyền Hình|
|NTFD93673|  KPLUS|         2288|Truyền Hình|
|HNJ063267|  KPLUS|         2282|Truyền Hình|
|HNH790383|  KPLUS|          906|Truyền Hình|
|THFD12466|  KPLUS|          242|T

In [9]:
df1 = df1.withColumn('Date',lit(date))

In [10]:
df1.show()

+---------+-------+-------------+-----------+----------+
| Contract|Appname|TotalDuration|       Type|      Date|
+---------+-------+-------------+-----------+----------+
|HNH579912|  KPLUS|          254|Truyền Hình|2022-04-01|
|HUFD40665|  KPLUS|         1457|Truyền Hình|2022-04-01|
|HNH572635|  KPLUS|         2318|Truyền Hình|2022-04-01|
|HND141717|  KPLUS|         1452|Truyền Hình|2022-04-01|
|HNH743103|  KPLUS|          251|Truyền Hình|2022-04-01|
|HNH893773|  KPLUS|          924|Truyền Hình|2022-04-01|
|HND083642|  KPLUS|         1444|Truyền Hình|2022-04-01|
|DNFD74404|  KPLUS|          691|Truyền Hình|2022-04-01|
|DTFD21200|  KPLUS|         1436|Truyền Hình|2022-04-01|
|LDFD05747|  KPLUS|         1434|Truyền Hình|2022-04-01|
|HNH063566|  KPLUS|          687|Truyền Hình|2022-04-01|
|HNH866786|  KPLUS|          248|Truyền Hình|2022-04-01|
|NBAAA1128|  KPLUS|          247|Truyền Hình|2022-04-01|
|HNH960439|  KPLUS|          683|Truyền Hình|2022-04-01|
|HNJ035736|  KPLUS|          24

In [11]:
A = df1.select('Contract','Type','TotalDuration')[df1.Type == "Truyền Hình"].show()

+---------+-----------+-------------+
| Contract|       Type|TotalDuration|
+---------+-----------+-------------+
|HNH579912|Truyền Hình|          254|
|HUFD40665|Truyền Hình|         1457|
|HNH572635|Truyền Hình|         2318|
|HND141717|Truyền Hình|         1452|
|HNH743103|Truyền Hình|          251|
|HNH893773|Truyền Hình|          924|
|HND083642|Truyền Hình|         1444|
|DNFD74404|Truyền Hình|          691|
|DTFD21200|Truyền Hình|         1436|
|LDFD05747|Truyền Hình|         1434|
|HNH063566|Truyền Hình|          687|
|HNH866786|Truyền Hình|          248|
|NBAAA1128|Truyền Hình|          247|
|HNH960439|Truyền Hình|          683|
|HNJ035736|Truyền Hình|          246|
|NTFD93673|Truyền Hình|         2288|
|HNJ063267|Truyền Hình|         2282|
|HNH790383|Truyền Hình|          906|
|THFD12466|Truyền Hình|          242|
|HNH566080|Truyền Hình|          242|
+---------+-----------+-------------+
only showing top 20 rows



In [23]:
df1.show()

+---------+-------+-------------+-----------+----------+
| Contract|Appname|TotalDuration|       Type|      Date|
+---------+-------+-------------+-----------+----------+
|HNH579912|  KPLUS|          254|Truyền Hình|2022-04-01|
|HUFD40665|  KPLUS|         1457|Truyền Hình|2022-04-01|
|HNH572635|  KPLUS|         2318|Truyền Hình|2022-04-01|
|HND141717|  KPLUS|         1452|Truyền Hình|2022-04-01|
|HNH743103|  KPLUS|          251|Truyền Hình|2022-04-01|
|HNH893773|  KPLUS|          924|Truyền Hình|2022-04-01|
|HND083642|  KPLUS|         1444|Truyền Hình|2022-04-01|
|DNFD74404|  KPLUS|          691|Truyền Hình|2022-04-01|
|DTFD21200|  KPLUS|         1436|Truyền Hình|2022-04-01|
|LDFD05747|  KPLUS|         1434|Truyền Hình|2022-04-01|
|HNH063566|  KPLUS|          687|Truyền Hình|2022-04-01|
|HNH866786|  KPLUS|          248|Truyền Hình|2022-04-01|
|NBAAA1128|  KPLUS|          247|Truyền Hình|2022-04-01|
|HNH960439|  KPLUS|          683|Truyền Hình|2022-04-01|
|HNJ035736|  KPLUS|          24

In [27]:
df1.select(sum('TotalDuration')).collect()

[Row(sum(TotalDuration)=26038134476)]

In [28]:
df1.agg({'TotalDuration': 'sum'}).collect()

[Row(sum(TotalDuration)=26038134476)]

In [25]:
df1[df1.Contract == 'HUFD07189'].show()

+---------+-------+-------------+-----------+----------+
| Contract|Appname|TotalDuration|       Type|      Date|
+---------+-------+-------------+-----------+----------+
|HUFD07189|  KPLUS|          276|Truyền Hình|2022-04-01|
|HUFD07189|CHANNEL|         1988|Truyền Hình|2022-04-01|
+---------+-------+-------------+-----------+----------+



In [14]:
df_cal = df1.select('Contract','Type','TotalDuration')
df_cal_group = df_cal.groupBy('Contract','Type').count()
df_cal_group.show()

+---------+-----------+-----+
| Contract|       Type|count|
+---------+-----------+-----+
|DNH014998|Phim Truyện|    1|
|HND486882|Phim Truyện|    1|
|HUFD07189|Truyền Hình|    2|
|HDFD36288|Truyền Hình|    2|
|CTFD04401|Truyền Hình|    2|
|HNH954607|Phim Truyện|    1|
|HNH855959|Truyền Hình|    2|
|SGH034683|Truyền Hình|    1|
|NTFD35330|Truyền Hình|    1|
|NTFD48198|Phim Truyện|    1|
|HNH443856|Truyền Hình|    2|
|NAFD05338|Truyền Hình|    1|
|LCFD20510|Phim Truyện|    1|
|QNFD29007|Truyền Hình|    1|
|SGH569599|Truyền Hình|    1|
|SGH701752|Truyền Hình|    2|
|HNH712164|Phim Truyện|    1|
|PYFD01920|Truyền Hình|    1|
|HTFD13716|Truyền Hình|    1|
|SGJ039473|Truyền Hình|    1|
+---------+-----------+-----+
only showing top 20 rows



In [16]:
df_cal_group_2 = df_cal_group.groupBy('Type').count().show()

+-----------+-------+
|       Type|  count|
+-----------+-------+
|   Thể Thao|   2292|
|Truyền Hình|1355367|
|  Thiếu Nhi|  33056|
|   Giải Trí|   6394|
|Phim Truyện|  86312|
+-----------+-------+



## Dùng RDD để tạo cột Type:
    RDD sẽ không tạo cột Type mà sẽ thay thế trực tiếp trên data gốc nhờ hàm lambda 

In [ ]:
# stateDic={'CHANNEL':'Truyền Hình','DSHD':'Truyền Hình', 'KPLUS':'Truyền Hình',
#           'VOD' : 'Phim truyen', 'FIMS': 'Phim truyen',
#           'SPORT': 'The thao', 'RELAX': 'Giai tri','CHILD': 'Thieu nhi'}
# stateDic['KPLUS']

'Truyền Hình'

In [ ]:
# df.show()

+--------------------+-------+------+--------------------+-----+
|                 _id| _index|_score|             _source|_type|
+--------------------+-------+------+--------------------+-----+
|AX_momhia1FFivsGrn9o|history|     0|{KPLUS, HNH579912...|kplus|
|AX_momhca1FFivsGrnvg|history|     0|{KPLUS, HUFD40665...|kplus|
|AX_momhaa1FFivsGrnny|history|     0|{KPLUS, HNH572635...|kplus|
|AX_momhca1FFivsGrnvv|history|     0|{KPLUS, HND141717...|kplus|
|AX_momhia1FFivsGrn98|history|     0|{KPLUS, HNH743103...|kplus|
|AX_momg9a1FFivsGrnkS|history|     0|{KPLUS, HNH893773...|kplus|
|AX_momhca1FFivsGrnwA|history|     0|{KPLUS, HND083642...|kplus|
|AX_momhfa1FFivsGrn2u|history|     0|{KPLUS, DNFD74404...|kplus|
|AX_momhca1FFivsGrnwP|history|     0|{KPLUS, DTFD21200...|kplus|
|AX_momhca1FFivsGrnwU|history|     0|{KPLUS, LDFD05747...|kplus|
|AX_momhfa1FFivsGrn24|history|     0|{KPLUS, HNH063566...|kplus|
|AX_momhia1FFivsGrn-W|history|     0|{KPLUS, HNH866786...|kplus|
|AX_momhia1FFivsGrn-a|his

In [ ]:
# df_rdd = df
# df_rdd.rdd.collect()
# df_rdd=df_rdd.rdd.map(lambda x: (x.Contract,stateDic[x.AppName], x.TotalDuration)).toDF(["Contract","AppName","TotalDuration" ])